In [1]:
from tqdm import tqdm
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
from WordPieceTokenizer import WordPieceTokenizer as Tokenizer
import TransferDataset
from TransferDataset import CustomSentimentDataset

DATA_DIRECTORY = "datasets/"
MODEL_DIRECTORY = "Pretrained/"
VOCAB_FILE_PATH = f"{MODEL_DIRECTORY}vocab.txt"
MAX_SEQUENCE_LENGTH = 150
BATCH_SIZE = 64

In [2]:
tokenizer = Tokenizer(vocab_file_path=VOCAB_FILE_PATH, do_lower_case=False,strip_accents=False,clean_text=True)
# model = BertForSequenceClassification.from_pretrained(MODEL_DIRECTORY,num_labels=7)

In [3]:
# <-- Load DataFrame -->
print("DataFrame 불러오기 : [시작]")
df = pd.read_csv(f"{DATA_DIRECTORY}Sentiment_data.csv",index_col=0)

df.loc[(df['감정'] == '불안'),'감정'] = 0
df.loc[(df['감정'] == '당황'),'감정'] = 1
df.loc[(df['감정'] == '분노'),'감정'] = 2
df.loc[(df['감정'] == '슬픔'),'감정'] = 3
df.loc[(df['감정'] == '중립'),'감정'] = 4
df.loc[(df['감정'] == '행복'),'감정'] = 5
df.loc[(df['감정'] == '혐오'),'감정'] = 6

train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['발화'],df['감정'],test_size=0.3, random_state=42,stratify=df['감정'])
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts,temp_labels,test_size=0.5, random_state=42,stratify=temp_labels)

print(f"총 데이터 수: {len(df)}")
print(f"학습 세트 수: {len(train_texts)}")
print(f"검증 세트 수: {len(val_texts)}")
print(f"테스트 세트 수: {len(test_texts)}")
print("DataFrame 불러오기 : [완료]")

# <-- Tokenization -->
print("토큰화 작업 : [시작]")
print("토큰화 작업 : [학습 데이터셋 토큰화 중]")
train_encodings = TransferDataset.tokenize_texts(list(train_texts), tokenizer, MAX_SEQUENCE_LENGTH)
print("토큰화 작업 : [검증 데이터셋 토큰화 중]")
val_encodings = TransferDataset.tokenize_texts(list(val_texts), tokenizer, MAX_SEQUENCE_LENGTH)
print("토큰화 작업 : [테스트 데이터셋 토큰화 중]")
test_encodings = TransferDataset.tokenize_texts(list(test_texts), tokenizer, MAX_SEQUENCE_LENGTH)
print("토큰화 작업 : [완료]")

# <-- Dataset & DataLoader -->
print("데이터셋 합치기 : [시작]")
train_dataset = CustomSentimentDataset(train_encodings, list(train_labels))
val_dataset = CustomSentimentDataset(val_encodings, list(val_labels))
test_dataset = CustomSentimentDataset(test_encodings, list(test_labels))
print("데이터셋 합치기 : [완료]")

print("DataLoader 변환 : [시작]")
train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset,batch_size=BATCH_SIZE, shuffle=False)
print("DataLoader 변환 : [완료]")

DataFrame 불러오기 : [시작]
총 데이터 수: 146348
학습 세트 수: 102443
검증 세트 수: 21952
테스트 세트 수: 21953
DataFrame 불러오기 : [완료]
토큰화 작업 : [시작]
토큰화 작업 : [학습 데이터셋 토큰화 중]


100%|███████████████████████████████████████████████████████████████████████| 102443/102443 [00:08<00:00, 12736.22it/s]


토큰화 작업 : [검증 데이터셋 토큰화 중]


100%|█████████████████████████████████████████████████████████████████████████| 21952/21952 [00:01<00:00, 11036.31it/s]


토큰화 작업 : [테스트 데이터셋 토큰화 중]


100%|█████████████████████████████████████████████████████████████████████████| 21953/21953 [00:01<00:00, 13158.43it/s]


토큰화 작업 : [완료]
데이터셋 합치기 : [시작]
데이터셋 합치기 : [완료]
DataLoader 변환 : [시작]
DataLoader 변환 : [완료]
